### 데이터 불러오기

In [28]:
import pandas as pd
import os

os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

df_tr = pd.read_csv("./data/raw/train.csv")


df_v = pd.read_csv("./data/raw/dev.csv")


df_tt = pd.read_csv("./data/raw/test.csv")

In [29]:
df_tr.sample(5)

,id,source,sentence_1,sentence_2,label,binary-label
6801,boostcamp-sts-v1-train-6801,nsmc-sampled,영원히이루어질수없게되버린 핀첼커플,여인은 오늘도 쳇바퀴속에서 벗어날수 없는 현실로부터의 도피를 꿈꾼다,0.0,0.0
4108,boostcamp-sts-v1-train-4108,nsmc-sampled,진짜 죽기전에 봐야할 걸작선~!,죽기 전에 꼭 봐야할 영화 선,3.8,1.0
6722,boostcamp-sts-v1-train-6722,slack-rtt,오.. 단순 회의 참가는 VR이 없어도 되네요.,아.. VR이 필요없는 간단한 회의참가.,3.8,1.0
3649,boostcamp-sts-v1-train-3649,slack-rtt,비행기 타기가 귀찮아서 여행 안가는 저에겐 신기할 따름.,비행이 귀찮아서 여행을 하지 않는 저에게는 기적입니다.,3.2,1.0
9316,boostcamp-sts-v1-train-9316,slack-sampled,어쩌다 보니 사람 들어간 사진은 못찍었는데 너무 맛있었고 재밌었습니다!!,너무 맛있어서 또 가고싶은집!!,1.6,0.0


### sentence 정제

In [30]:
import re
from pykospacing import Spacing

spacing = Spacing()


def clean_text(text):

    # 쓸모없는 이모티콘(ㅋ, ㅎ, ㅇ, ㄷ 등) 제거
    text = re.sub(r"[ㅋㅎㄷㅇ]+", "", text)

    # 특수문자 제거 (문장부호는 남김)
    text = re.sub(r"[^가-힣a-zA-Z0-9.,!? ]+", "", text)

    # 불필요한 공백 제거
    text = re.sub(r"\s+", " ", text).strip()

    # 문장 속 .가 3개 이상인 경우 ...로 변경
    text = re.sub(r"\.{4,}", "...", text)

    # 문장 속 ,가 3개 이상인 경우 ,,,로 변경
    text = re.sub(r"\,{4,}", ",,,", text)

    # 문장 속 !가 3개 이상인 경우 !로 변경
    text = re.sub(r"!{4,}", "!", text)

    # 문장 속 ?가 3개 이상인 경우 ???로 변경
    text = re.sub(r"\?{4,}", "?", text)

    # 문장 속 ㅜ가 3개 이상인 경우 ㅠㅠ로 변경
    text = re.sub(r"ㅜ{1,}", "ㅠ", text)

    # 문장 속 ㅠ가 3개 이상인 경우 ㅠㅠ로 변경
    text = re.sub(r"ㅠ{3,}", "ㅠㅠ", text)

    return text


def processing(df):
    df["sentence_1"] = df["sentence_1"].apply(spacing)
    df["sentence_2"] = df["sentence_2"].apply(spacing)

    # 문장 변경
    df["sentence_1"] = df["sentence_1"].apply(clean_text)
    df["sentence_2"] = df["sentence_2"].apply(clean_text)

    if "sentence_1_clean" in df.columns:
        df.drop(columns=["sentence_1_clean"], inplace=True)

    if "sentence_2_clean" in df.columns:
        df.drop(columns=["sentence_2_clean"], inplace=True)

    return df

In [31]:
df_tr = processing(df_tr)
df_tr.to_csv("./data/custom/train_v1.0.5_clean_spacing.csv", index=False)


df_v = processing(df_v)
df_v.to_csv("./data/custom/dev_v1.0.5_clean_spacing.csv", index=False)


df_tt = processing(df_tt)
df_tt.to_csv("./data/custom/test_v1.0.5_clean_spacing.csv", index=False)

In [19]:
def check_data_types(df):
    # 각 컬럼의 데이터 타입 확인
    print(df[["sentence_1", "sentence_2"]].dtypes)

In [23]:
check_data_types(df_tr)

sentence_1    object
sentence_2    object
dtype: object


### 문맥을 고려한 텍스트 데이터 증강

In [34]:
import transformers
import re
import random
import numpy as np


class BERT_Augmentation:
    def __init__(self):
        self.model_name = "monologg/koelectra-base-v3-generator"
        self.model = transformers.AutoModelForMaskedLM.from_pretrained(self.model_name)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)
        self.unmasker = transformers.pipeline(
            "fill-mask", model=self.model, tokenizer=self.tokenizer, device=0
        )
        random.seed(42)

    # 랜덤하게 토큰? 글자에 마스크 씌우기
    def random_masking_replacement(self, sentence, ratio=0.3):
        """Masking random eojeol of the sentence, and recover them using PLM.

        Args:|
            sentence (str): Source sentence
            ratio (int): Ratio of masking

        Returns:
          str: Recovered sentence
        """

        span = int(round(len(sentence.split()) * ratio))

        # 품질 유지를 위해, 문장의 어절 수가 4 이하라면 원문장을 그대로 리턴합니다.
        if len(sentence.split()) <= 4:
            return sentence

        mask = self.tokenizer.mask_token
        unmasker = self.unmasker

        unmask_sentence = sentence
        # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음.
        random_idx = random.randint(1, len(unmask_sentence.split()) - span)

        unmask_sentence = unmask_sentence.split()
        # del unmask_sentence[random_idx:random_idx+span]
        cache = []
        for _ in range(span):
            # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음.
            while cache and random_idx in cache:
                random_idx = random.randint(1, len(unmask_sentence) - 2)
            cache.append(random_idx)
            unmask_sentence[random_idx] = mask
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]["sequence"]
            unmask_sentence = unmask_sentence.split()
        unmask_sentence = " ".join(unmask_sentence)
        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

    def random_masking_insertion(self, sentence, ratio=0.3):

        span = int(round(len(sentence.split()) * ratio))
        mask = self.tokenizer.mask_token
        unmasker = self.unmasker

        # Recover
        unmask_sentence = sentence

        for _ in range(span):
            unmask_sentence = unmask_sentence.split()
            random_idx = random.randint(0, len(unmask_sentence) - 1)
            unmask_sentence.insert(random_idx, mask)
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]["sequence"]

        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

In [35]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import random
import requests
from kiwipiepy import Kiwi
import time
from quickspacer import Spacer


class AdverbAugmentation:
    def __init__(self):
        self.kiwi = Kiwi()
        self.spacing = Spacer().space
        self.gloss_cache = {}  # 캐시 추가

    def _adverb_detector(self, sentence):

        # POS info
        pos_list = [(x[0], x[1]) for x in self.kiwi.tokenize(sentence)]  # (token, pos)

        adverb_list = []
        for pos in pos_list:
            if pos[1] == "MAG" and len(pos[0]) > 1:  # 1음절 부사는 제외함.
                adverb_list.append(pos[0])
        return adverb_list

    def _get_gloss(self, word):
        if word in self.gloss_cache:  # 캐시 확인
            return self.gloss_cache[word]

        res = requests.get("https://dic.daum.net/search.do?q=" + word, timeout=5)
        time.sleep(random.uniform(0.5, 2.5))
        soup = BeautifulSoup(res.content, "html.parser")
        try:
            # 첫 번째 뜻풀이.
            meaning = soup.find("span", class_="txt_search")
        except AttributeError:
            return word
        if not meaning:
            return word

        # parsing 결과에서 한글만 추출
        meaning = re.findall("[가-힣]+", str(meaning))
        meaning = " ".join(meaning)

        # 띄어쓰기 오류 교정 (위 에 -> 위에)
        # meaning = spell_checker.check(meaning).as_dict()['checked'].strip()
        meaning = self.spacing([meaning.replace(" ", "")])

        self.gloss_cache[word] = meaning[0].strip()  # 캐시에 저장

        return meaning[0].strip()

    def adverb_gloss_replacement(self, sentence):
        adverb_list = self._adverb_detector(sentence)
        if adverb_list:
            # 부사들 중에서 1개만 랜덤으로 선택합니다.
            adverb = random.choice(adverb_list)
            try:
                gloss = self._get_gloss(adverb)
                sentence = sentence.replace(adverb, gloss)
            except:
                print("except: ", sentence)
                pass
        return sentence

In [36]:
import random
import pickle

wordnet = {}
with open("./data/custom/wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)


def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    if len(new_words) != 0:
        sentence = " ".join(new_words)
        new_words = sentence.split(" ")

    else:
        new_words = ""

    return new_words


def get_synonyms(word):
    synomyms = []

    try:
        for syn in wordnet[word]:
            for s in syn:
                synomyms.append(s)
    except:
        pass

    return synomyms

In [45]:
df_tr[df_tr["label"] > 0]

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도 있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,"반전도 있고, 사랑도 있고 재미도 있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근 권한이 없다고 뜹니다,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약 조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사 후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다 가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
5,boostcamp-sts-v1-train-005,nsmc-rtt,오마이가 뜨지져 스크롸이 스트휏,오 마이 갓 지저스 스크론 이스트 팬,2.6,1.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야 합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합 니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급 좀 줄여주세요,공무원 봉급 좀 줄이지 좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못 먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [37]:
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
import asyncio


# 데이터 로드
df = df_tr

original_data = df.copy()

# BERT 증강 및 부사 치환 증강 클래스 인스턴스 생성
BERT_aug = BERT_Augmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion
adverb_aug = AdverbAugmentation()
adverb_gloss_replacement = adverb_aug.adverb_gloss_replacement


# 증강 함수 정의
def augment_sentence(row, aug_method):
    row["sentence_1"] = aug_method(row["sentence_1"])
    row["sentence_2"] = aug_method(row["sentence_2"])
    return row


# 각 증강 기법에 따라 데이터셋 증강
def augment_dataset(df, aug_method, source_label, n_jobs=1, num_aug=1):
    augmented_rows = []
    for _ in range(num_aug):  # 각 문장에 대해 num_aug 만큼 증강
        augmented_rows += Parallel(n_jobs=n_jobs)(
            delayed(augment_sentence)(row.copy(), aug_method)
            for idx, row in tqdm(df.iterrows(), total=len(df))
        )
    augmented_df = pd.DataFrame(augmented_rows)
    augmented_df["source"] = source_label  # 증강 방법에 따라 source 설정
    return augmented_df


# 증강 기법 적용

# 1. Random Masking Insertion (BERT)
print("Random Masking Insertion 중...")
masking_insertion_augset = augment_dataset(
    df.copy(), lambda x: random_masking_insertion(x, ratio=0.15), "data-argu-RMI"
)


# 2. 부사 치환
print("부사 치환 중...")
adverb_augset = augment_dataset(df.copy(), adverb_gloss_replacement, "data-argu-adverb")


# 원본 데이터와 결합 및 중복 제거
augmented_data = pd.concat([masking_insertion_augset, adverb_augset])
augmented_data = augmented_data.drop_duplicates(
    subset=["sentence_1", "sentence_2"]
).reset_index(drop=True)

# 증강 데이터에 id 업데이트
augmented_data["id"] = ["argu-{:03d}".format(i) for i in range(len(augmented_data))]

# 원본 데이터와 결합 및 중복 제거
final_data = (
    pd.concat([original_data, augmented_data])
    .drop_duplicates(subset=["sentence_1", "sentence_2"])
    .reset_index(drop=True)
)

# 결과 저장
final_data.to_csv("./data/custom/train_v1.4.5_cl_argu.csv", index=False)

C:\Users\Yohan\.virtualenvs\level1-semantictextsimilarity-nlp-14-Qc70mCay\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Random Masking Insertion 중...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9324/9324 [03:02<00:00, 51.15it/s]


부사 치환 중...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9324/9324 [14:07<00:00, 11.01it/s]


In [39]:
final_data = final_data[final_data["source"] != "data-argu-adverb"]

final_data.to_csv("./data/custom/train_v1.4.5_cl_argu_noadverb.csv", index=False)